## Steps

### Import libraries

Here we will import the libraries needed for this notebook.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
import os
import pandas as pd
import json
from IPython.display import display

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)

In [2]:
wallaroo.__version__

'2023.1.0+9c085b4a'

### Arrow Support

As of the 2023.1 release, Wallaroo provides support for dataframe and Arrow for inference inputs.  This tutorial allows users to adjust their experience based on whether they have enabled Arrow support in their Wallaroo instance or not.

If Arrow support has been enabled, `arrowEnabled=True`. If disabled or you're not sure, set it to `arrowEnabled=False`

The examples below will be shown in an arrow enabled environment.

In [3]:
import os
# Only set the below to make the OS environment ARROW_ENABLED to TRUE.  Otherwise, leave as is.
# os.environ["ARROW_ENABLED"]="True"

if "ARROW_ENABLED" not in os.environ or os.environ["ARROW_ENABLED"].casefold() == "False".casefold():
    arrowEnabled = False
else:
    arrowEnabled = True
print(arrowEnabled)

False


### Connect to the Wallaroo Instance

This command will be used to set up a connection to the Wallaroo cluster and allow creating and use of Wallaroo inference engines.

In [4]:
# Client connection from local Wallaroo instance

wl = wallaroo.Client()

# SSO login through keycloak

# wallarooPrefix = "squishy-wallaroo-6187"
# wallarooSuffix = "wallaroo.dev"

# wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
#                     auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
#                     auth_type="sso")

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace for all other commands.

To allow this tutorial to be run multiple times or by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.

In [5]:
workspace_name = 'houseprice-abtesting-nbz'

In [6]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

In [7]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

{'name': 'houseprice-abtesting-nbz', 'id': 101, 'archived': False, 'created_by': '3f1009e8-9726-4c1e-8e12-ee16cf375679', 'created_at': '2023-03-13T23:13:46.247093+00:00', 'models': [{'name': 'housing-control', 'versions': 4, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 3, 14, 0, 32, 44, 92612, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 3, 13, 23, 19, 59, 151078, tzinfo=tzutc())}, {'name': 'housing-challenger', 'versions': 4, 'owner_id': '""', 'last_update_time': datetime.datetime(2023, 3, 14, 0, 32, 46, 475364, tzinfo=tzutc()), 'created_at': datetime.datetime(2023, 3, 13, 23, 20, 2, 633197, tzinfo=tzutc())}], 'pipelines': [{'name': 'randomsplitpipeline-demo', 'create_time': datetime.datetime(2023, 3, 13, 23, 20, 24, 61852, tzinfo=tzutc()), 'definition': '[]'}]}

### Set Up the Champion and Challenger Models

Now we upload the Champion and Challenger models to our workspace.  We will use two models:

1. a random forest model `rf_model.onnx` for the champion
2. an xgboost model `xgb_model.onnx` for the challenger

### Set the Champion Model

We upload our champion model, labeled as `control`.

In [8]:
control =  wl.upload_model("housing-control",   'models/rf_model.onnx').configure()

### Set the Challenger Model

Now we upload the Challenger model, labeled as `challenger`.

In [9]:
challenger = wl.upload_model("housing-challenger",   'models/xgb_model.onnx').configure()

### Define The Pipeline

Here we will configure a pipeline with two models and set the control model with a random split chance of receiving 2/3 of the data.  Because this is a random split, it is possible for one model or the other to receive more inferences than a strict 2:1 ratio, but the more inferences are run, the more likely it is for the proper ratio split.

In [10]:
pipeline = (wl.build_pipeline("randomsplitpipeline-demo")
            .add_random_split([(2, control), (1, challenger)], "session_id"))

### Deploy the pipeline

Now we deploy the pipeline so we can run our inference through it.

In [11]:
experiment_pipeline = pipeline.deploy()

Waiting for deployment - this will take up to 45s ............. ok


In [12]:
experiment_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.64.2.197',
   'name': 'engine-5779ddc65c-jxxqg',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'randomsplitpipeline-demo',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'housing-control',
      'version': '33301a85-9982-4ecc-9602-ee9773ccae38',
      'sha': 'e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6',
      'status': 'Running'},
     {'name': 'housing-challenger',
      'version': 'c54abb74-3764-4540-b940-ff139e8845ac',
      'sha': '31e92d6ccb27b041a324a7ac22cf95d9d6cc3aa7e8263a229f7c4aec4938657c',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.64.2.198',
   'name': 'engine-lb-74b4969486-2wr8h',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

# Run a single inference
Now we have our deployment set up let's run a single inference. In the results we will be able to see the inference results as well as which model the inference went to under model_id.  We'll run the inference request 5 times, with the odds are that the challenger model being run at least once.

In [14]:
results = []
if arrowEnabled is True:
    # use dataframe JSON files
    for x in range(1):
        result = experiment_pipeline.infer_from_file("data/xtest-1.df.json")
        display(result.loc[:,["out.split", "out.main"]])        
else:
    # use Wallaroo JSON files
    results.append(experiment_pipeline.infer_from_file("data/xtest-1.json"))
    results.append(experiment_pipeline.infer_from_file("data/xtest-1.json"))
    results.append(experiment_pipeline.infer_from_file("data/xtest-1.json"))
    results.append(experiment_pipeline.infer_from_file("data/xtest-1.json"))
    results.append(experiment_pipeline.infer_from_file("data/xtest-1.json"))
    for result in results:
        print(result[0].model())
        print(result[0].data())

('housing-challenger', 'c54abb74-3764-4540-b940-ff139e8845ac')
[array([[659806.]]), array([[7.2147857e-313]])]
('housing-challenger', 'c54abb74-3764-4540-b940-ff139e8845ac')
[array([[659806.]]), array([[7.2147857e-313]])]
('housing-control', '33301a85-9982-4ecc-9602-ee9773ccae38')
[array([[718013.6875]]), array([[7.2147857e-313]])]
('housing-control', '33301a85-9982-4ecc-9602-ee9773ccae38')
[array([[718013.6875]]), array([[7.2147857e-313]])]
('housing-control', '33301a85-9982-4ecc-9602-ee9773ccae38')
[array([[718013.6875]]), array([[7.2147857e-313]])]


### Run Inference Batch

We will submit 1000 rows of test data through the pipeline, then loop through the responses and display which model each inference was performed in.  The results between the control and challenger should be approximately 2:1.

In [17]:
l = []
responses =[]

if arrowEnabled is True:
    #Read in the test data as one dataframe
    test_data = pd.read_json('data/xtest-1k.df.json')
    # For each row, submit that row as a separate dataframe
    # Add the results to the responses array
    for index, row in test_data.head(1000).iterrows():
        responses.append(experiment_pipeline.infer(row.to_frame('tensor').reset_index(drop=True)))
    #now get our responses for each row
    # each r is a dataframe, then get the result from out.split into json and get the model name
    l = [json.loads(r.loc[0]["out.split"][0])["name"] for r in responses]
    df = pd.DataFrame({'model': l})
    display(df.model.value_counts())
else:
    from data import test_data
    for nth in range(1000):
        responses.extend(experiment_pipeline.infer(test_data.data[nth]))
    l = [r.raw['model_name'] for r in responses]
    df = pd.DataFrame({'model': l})
    display(df.model.value_counts())


housing-control       658
housing-challenger    342
Name: model, dtype: int64

### Test Challenger

Now we have run a large amount of data we can compare the results.

For this experiment we are looking for a significant change average predicted price between the two models.

In [18]:
control_count = 0
challenger_count = 0

control_sum = 0
challenger_sum = 0

if arrowEnabled is True:
    # do nothing
    for r in responses:
        if json.loads(r.loc[0]["out.split"][0])["name"] == "housing-control":
            control_count += 1
            control_sum += r.loc[0]["out.main"][0]
        else:
            challenger_count += 1
            challenger_sum += r.loc[0]["out.main"][0]
else:
    for r in responses:
        if r.raw['model_name'] == "housing-control":
            control_count += 1
            control_sum += r.raw['outputs'][0]['Float']['data'][0]
        else:
            challenger_count +=1
            challenger_sum += r.raw['outputs'][0]['Float']['data'][0]
           
            
print("control mean price prediction: " + str(control_sum/control_count))
print("challenger mean price prediction: " + str(challenger_sum/challenger_count))

control mean price prediction: 536026.0709773937
challenger mean price prediction: 549229.8949195907


### Undeploy Pipeline

With the testing complete, we undeploy the pipeline to return the resources back to the environment.

In [20]:
experiment_pipeline.undeploy()

Waiting for undeployment - this will take up to 45s ............................................

RuntimeError: Undeployment did not finish within 45s.
Status: None

In [21]:
experiment_pipeline

name,randomsplitpipeline-demo
created,2023-03-13 23:20:24.061852+00:00
last_updated,2023-03-14 00:45:12.882103+00:00
deployed,False
tags,
versions,"9d30ef2e-9bbf-46fb-b0c6-a3b9bd7fda52, 6a902bd2-12fc-4990-8659-1409cc5dd2ef, 988a557f-7295-45f2-9b85-32eb78b20e32, 318a3608-e80f-4a0c-a394-bdf668d8a28b, 30566928-1088-4cae-a02f-19b910c15690, 6021caad-646d-4357-b0a4-f0cdaaf19ce3, 485e6a27-fb20-488d-9d7a-a8a374c0bd96, 4c6423ff-a5d2-4e02-b45e-718d02f1049c, 15158f58-a345-4342-b381-c8378a829837, e760b639-2d05-4c06-821f-3ab944f0040a, 89d9c6bf-b110-4712-a902-88d5b2bb2a9e, 1863e003-8999-40d2-a23f-31be6c39c40d"
steps,housing-control
